<a href="https://colab.research.google.com/github/thamonS/bcs_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [1]:
import pandas as pd
import os

##บ้านมือสอง

In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

In [34]:
gc = gspread.authorize(creds)

spreadsheet_key1 = '1_0Lvo1DH2yOZ5stmFIBYmyPGVLX0I-70NVXkP2lyxTw'
sh1 = gc.open_by_key(spreadsheet_key1)

worksheet_sechouse = 'บ้านมือ2'
sechouse = sh1.worksheet(worksheet_sechouse)

sechouse_data = sechouse.get_all_values()
df_second_house = pd.DataFrame(sechouse_data[1:], columns=sechouse_data[0])
df_second_house.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,,,,,,,,,
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,,,,,,,,,
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,,,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,


In [56]:
df_second_house.columns

Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden'],
      dtype='object')

In [35]:
columns1 = ['renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom',
            'direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 'sub_district', 'zone']

df_second_house1 = df_second_house[columns1]
df_second_house1.head()

,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone
0,no,0,1,45,2,2,2,ไม่ระบุ,ไม่มี,244,2,390000,ชนบท,ศรีบุญเรือง,
1,no,0,0,88,1,1,1,ไม่ระบุ,ไม่มี,164,3,820000,น้ำพอง,ทรายมูล,
2,no,1,0,8,1,11,11,ไม่ระบุ,ไม่ระบุ,489,9,3260000,มัญจาคีรี,กุดเค้า,
3,no,0,0,61.6,2,3,3,ไม่ระบุ,ไม่ระบุ,203,2,5000000,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
4,no,0,0,72.3,2,3,2,ไม่ระบุ,ไม่ระบุ,164,2,3500000,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน


In [37]:
df_second_house1.columns

Index(['renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va',
       'number_of_story', 'number_of_bedroom', 'number_of_bathroom',
       'direction', 'furniture', 'usable_area', 'number_of_parking',
       'sell_price', 'district', 'sub_district', 'zone'],
      dtype='object')

##7-11 and school

###import

In [ ]:
import json

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
f1 = open('/content/drive/MyDrive/bsc_dpdm24/data/MIDterm/7-eleven.json')
json_711 = json.load(f1)
f1.close()

In [32]:
f2 = open('/content/drive/MyDrive/bsc_dpdm24/data/MIDterm/school.json')
json_school = json.load(f2)
f2.close()

In [25]:
first_key = next(iter(json_school))
print(json_school[first_key])

[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 16.1311875, 'lng': 102.6504375}, 'viewport': {'northeast': {'lat': 16.13271247989272, 'lng': 102.6517733298927}, 'southwest': {'lat': 16.13001282010728, 'lng': 102.6490736701073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/school-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/school_pinlet', 'name': 'Ban La Wa Health Center', 'photos': [{'height': 3456, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109948283369748538605">A Google User</a>'], 'photo_reference': 'ARywPAKrl4GkxA4eldDQLd4bLsqag1nq-g5lacaHgemCiE3WPI4F53bqs0YxmhADKZMTh7Q_q-DsUUqNoYE6cCM8bVRfOXLaLqO2b6y97rIPMho8U7WwZPTg5LEdwjCRULM_5AkIgaxlYSSYfCgULGHDtro2GPqPC99E-JjwFCjz35fYkwqM', 'width': 4608}], 'place_id': 'ChIJW5GGVOuCGDERn2dgufu1KOU', 'rating': 4.5, 'reference': 'ChIJW5GGVOuCGDERn2dgufu1KOU', 'scope': 'GOOGLE', 'types': ['school',

###จำนวน7-11 กับ โรงเรียน

In [43]:
school_locations = []
for key, schools in json_school.items():
    for school in schools:
        school_locations.append({
            'place_id': school['place_id'],
            'name': school['name'],
            'latitude': school['geometry']['location']['lat'],
            'longitude': school['geometry']['location']['lng']
        })
df_schools = pd.DataFrame(school_locations)

In [44]:
df_schools.head()

,place_id,name,latitude,longitude
0,ChIJW5GGVOuCGDERn2dgufu1KOU,Ban La Wa Health Center,16.131187,102.650437
1,ChIJ1QvkXLaCGDERc-lFOXnj0NY,Ban Wang Woen Kut Lom School,16.135064,102.629047
2,ChIJ3x63JVOdGDERRmIYy0FvMus,ศูนย์พัฒนาเด็กเล็กบ้านหนองหัวช้าง,16.148207,102.620759
3,ChIJG5AU2QCdGDERoKsuWds9l1w,บ้านหนองหวาย,16.127920,102.613829
4,ChIJ3z46bb7wIjERqVRwLwU9thY,Ban Nong Bua Ban School,16.674452,102.918958


In [54]:
seven11_locations = []
for key, value in json_711.items():
    for item in value:
        seven11_locations.append({
            'place_id': item['place_id'],
            'name': item['name'],
            'latitude': item['geometry']['location']['lat'],
            'longitude': item['geometry']['location']['lng']
        })
df_711 = pd.DataFrame(seven11_locations)

In [55]:
df_711.head()

,place_id,name,latitude,longitude
0,ChIJzbBjEk2BGDER4YEKqD5kwlI,7-Eleven,16.060315,102.726486
1,ChIJWwEGw7KGGDERygXhoovc3Hg,7-Eleven,16.056474,102.726844
2,ChIJa8T1gu-BGDERq8cyG-vAu_o,7-Eleven,16.059541,102.729431
3,ChIJyyeKOR6BGDERawb3IXZz_5U,7-Eleven สาขา PTTOR อำเภอบ้านไผ่ (14179),16.075200,102.735602
4,ChIJ9zwFukeBGDERFV0i5_PJNLc,7-Eleven สาขา ชุมชนหนองลุมพุก (06323),16.070648,102.734405


In [58]:
import numpy as np
from geopy.distance import geodesic

# Define a function to calculate the number of nearby locations within a radius
def count_nearby_locations(lat, lng, locations_df, radius_km):
    count = 0
    for _, row in locations_df.iterrows():
        location_coords = (row['latitude'], row['longitude'])
        house_coords = (lat, lng)
        distance = geodesic(house_coords, location_coords).kilometers
        if distance <= radius_km:
            count += 1
    return count



In [60]:
# Add columns for counts of 7-11 and schools within 1 km
df_second_house['d1000-7/11'] = df_second_house.apply(
    lambda row: count_nearby_locations(
        row['latitude'], row['longitude'], df_711, radius_km=1),
    axis=1
)

df_second_house['d1000-school'] = df_second_house.apply(
    lambda row: count_nearby_locations(
        row['latitude'], row['longitude'], df_schools, radius_km=1),
    axis=1
)

# Replace missing values with 0 (if there are houses without any nearby locations)
df_second_house['d1000-7/11'] = df_second_house['d1000-7/11'].fillna(0)
df_second_house['d1000-school'] = df_second_house['d1000-school'].fillna(0)

# Display results
print(df_second_house)


KeyboardInterrupt: 

In [ ]:
'd1000-7/11' และ 'd1000-school

##คะแนนนักประเมิน

In [77]:
gc = gspread.authorize(creds)

spreadsheet_key2 = '1zQBLpkZ0Zm460FopaxPIiZ3A2Op1q2WFjD8v8T4sdjw'
sh2 = gc.open_by_key(spreadsheet_key2)

worksheet_ID = 'บ้าน'
ID = sh2.worksheet(worksheet_ID)

ID_data = ID.get_all_values()
df_ID = pd.DataFrame(ID_data[1:], columns=ID_data[0])
df_ID.head()

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,500,000","5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,700,000","โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,200,000","5,000-15,000",,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,400,000","ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,"6,050,000","5,000,000","5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [64]:
df_ID.isnull().any()

,0
ID ผู้ประเมิน,False
post id,False
ID,False
property_type,False
ความมั่นใจในการเมินราคาทรัพย์,False
ราคาทรัพย์,False
ประเมินราคาซื้อ-ขาย,False
ปัจจัยในการประเมินราคาซื้อ-ขาย,False
เหตุผลไม่มั่นใจราคาประเมิน,False
ให้คะแนนทรัพย์,False


In [83]:
df_ID.columns

Index(['ID ผู้ประเมิน', 'post id', 'ID', 'property_type',
       'ความมั่นใจในการเมินราคาทรัพย์', 'ราคาทรัพย์', 'ประเมินราคาซื้อ-ขาย',
       'ปัจจัยในการประเมินราคาซื้อ-ขาย', 'เหตุผลไม่มั่นใจราคาประเมิน',
       'ให้คะแนนทรัพย์'],
      dtype='object')

In [82]:
df_ID1 = df_ID[df_ID['ID ผู้ประเมิน']=='1']
df_ID1.head()

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,500,000","5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,200,000","5,000-15,000",,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,"6,050,000","5,000,000","5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
6,1,https://nayoo.co/khonkaen/posts/564,564,บ้านเดี่ยว,มั่นใจ,"4,400,000","3,500,000","5,000-18,000",,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
8,1,https://nayoo.co/khonkaen/posts/566,566,บ้านเดี่ยว,มั่นใจ,"2,750,000","2,500,000","5,000-25,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [86]:
df_ID1['estimator'] = df_ID1['ประเมินราคาซื้อ-ขาย']

<ipython-input-86-91dfae464575>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ID1['estimator'] = df_ID1['ประเมินราคาซื้อ-ขาย']


In [87]:
df_ID1['score'] = df_ID1['ให้คะแนนทรัพย์']

<ipython-input-87-fa372af6fd06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ID1['score'] = df_ID1['ให้คะแนนทรัพย์']
